# Importing Dependecies:

In [11]:
# Importing dependecies:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
import os

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Importing Data:

In [2]:
df = pd.read_json('data/recipes_raw_nosource_fn.json')
df_1 = pd.read_json('data/recipes_raw_nosource_epi.json')
df_2 = pd.read_json('data/recipes_raw_nosource_ar.json')
df.head()

,p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,kRBQSWtqYWqtkb34FGeenBSbC32gIdO,PmoGGX9RC5zbMJXtZH1SKzWKLFX4Aii,g2RYP1spIUlAYsytDMsdfLNQEOnLUrO,94NWE4F4C5NzgH0zYqSdmPM/G55BjKO,HdIbPB/8cgk17hAzbc2jUN4VBn//QKS,mC7Nb4F2S.DMCjZ39kzS/ak9GqEp4iO,QaFfhzgVliz840HdAnIypaGteQn5q3K,2ZlbPBIZEDWBdGFcJ/KhApHyOTYvt.G,8zGobobyDsaZYEw2dC0aOx5OMhA.TK6,GFhRq1LHg7bITZxzrQAdN1vt4U/9AXq,1NTEgJqoZSeiOR3lIYJpY6MdJy8J9oe,bB.av.HvROUULt52UwCNhYjmFwFMcEe,R1AQFiWULG/FuRSEB/.qFwcAbj.Phu.,RrewJ0.1fBlHmLH3BaXji7y2D08.KKK,z8enrwKc4Z3gxVmuxOKMZI/6GKSt1N2,pKpcLCAbXuvJJA3IZVlEhNjhOsBw15K,1Yls1UOBlB8tCWEQBJud.hEpzr15KIC,MoNU9eS8AI7BA07L/NbQkMgMdAhU43W,0HTC6/JWf4bJnNhpnYq8wqdew1bjJfO,HhWW9P3PtcVFgJa0A/ILP1QkY9u.XV2,6EcFqHg6GCifvTAJ/wqdNPIQJKhpyLi,WGZp7rKLW7ZheJIAtz/P8ZRzz7mqmlS,q68RErl7vzM2apk72GE/pEn0CVOEZry,esOZ0jNZz7beIh14E0KO/W8x/K/jRve,HwwWzw4M8OFnirNJvatbCn1k8TAtHc.,5t9WTRW1LmAZijA8EHHZ9v5LAhGfDOu,3sIf1qZuDOUV.tbEkqvAZGRfGLlTCgC,V5GdC6vvg9J71EWy50B55LojJoBZMjG,k3nUcqWBqw04GVwcooooX4QgMIhNObC,Sr5NOGRwFL2Et29RpmmnmgggA.9QAHK,QSFgCm1C3roLQYIYcnNmQHcKv11woyG,USvt1m2a7RwPmcqzD0/Au3VIrhP1RfS,h.BGRIoIdDmHwh72bF0aoCOluCXKkqW,wosCcAjyrLqlGtKyeu3SrclW91lI8n.,WPeQwcG9YD7MZrMQ4FdAlRS5LkZAdrC,zPd4EpB9aJLKw7nDkKoLv68L2kyNVU2,ygpfQqCl4qS.B5rEE3XBmCfJcqY3KMu,51qro2lQOQbj.oGlsu9NbnQF83QjUuK,JyR61cAN81XDUH0elFITpIuPO0Wwnqa,wg7Frz5wokYazNqF7awKz/s/iXZX/5e,/qlwIX5k7wiNvn/1FTAH0QbLp2BVsWi,BOj.9Z7aA2FJTIKXqDuyPCVoWr9goAq,Ayq3syHvYUYoss1fz2Wv.UvyHUIN3wq,73ftuME6qPNdYWIlaky2J0XOJOvAMJu,ZP4/HHU4BtVBSfMUrUekTgSuFWdQEHC,xcjKuDBYLjAZU2Lc.xx7JHP5uwlh9.6,...,lzaOMqkwdxVjuOxLKVVVqiferVjhYKC,sXkr7XqYsvDLQzf3iPQOUU3pn0XO0hu,HrBzYP6FYOeIz36c7JyHdDxCj.EUs3u,G1druF7Jjry2yp4ALb8yrujeH5E4Sgu,sbWsONW8oqRO1CeH0XoRb1IrWF6lR66,wBHuOMyXrxWIGAhqPpZq8gYbYsECPge,V9HttwxFZ9aWluciKbhkVL.gXih9plu,MaZr1vLXiaUCaD6kxNQVX6NADIJxV7e,1QTk67BlaayW/PDb.e4L1imW0h0k79.,PxzSmYV76PI3ZZhrcFVC.z4SRx/WTAa,mkLvC4mucuxz8HznaZnv35u47XLCuOW,KsSDe3Imt60kFl7HeXFtqTnFaQ6lH7y,uhyQziyPlQm9.M58TlwQRljNoYVbEka,MYv2s2yDC7y5bAdY/Ebx7DDOY1NCjsK,Hyvs81Px7TVnyHEEhAiJJxjuviBdfR6,bpz9N2jObyJztLtzDhbDmnhX7.A3hC2,JhEGtpTV3uK89R3CxJ/0OPCIFewbeXG,RbtaeWxrrbVorgoJ0r6h0Rfi..sXQey,zhbWtPvCCzqLJOrXGe3FFBKwOTBmgOa,iK1IfL2jdQ60IqURoDdmzJU9XBhWLPW,1qL/FdX8S8.yaPQeSemXK0s2YCd5DdK,287hbwi.mjELP3k7q7vye7wMiN4Wh1W,.OnpU7Sv4uhZKWqG8QQBMqpzc92.pM.,U3i6cq5uT4M9Txzbn9IuOaQiUIPw3hW,MQAS8Dkmou9z.lp1MUvzVOdewGNL712,O3biElBi6G64bACtflU/ieI5TTQH8PW,w9X11wHJW14wVago.hufHUbC3rc/5CK,T2e30s7InFsyB4AVwNO4PMKt7FoweRW,IAxuGiMpvSJJhM0CIAt4r0NA9QYdv3y,FztHcktuaHKx7c6iN9Wz3wglKR.Djb2,SbuzjJ2X59.G47dIGzSilzuBCdLKeWm,/O6xDCwFukk7Tjy.wlc3z2xCjo688d.,M1gsyE32rg9BZjbEr9iYyr0bgq9ZedW,WKttZAMEU7.UfktDmxovEh8990RPKdm,CKug/Ejc6CP9FttAUIwBb65EKel43Eu,3R1S2jJXzb7ZPRV1KqXqolDvTcPrCUO,fxCsQzUFds4E.o4os3oZKd0rMNXqaNG,IQw/IdK0RwsP3HfPg9RyB.oRrBVBwwW,20Ymhyrxi/SkQBALrwWa.PqF6n.dto6,SkW5b1Rx.510iURG2vRz18u/dNUhZ62,CUZuiIs2F5A4u4EzEXB/yHN38So5CRy,08JThnwO3qzkhSKVuef6Ehg.HVPXt7C,0jaoQnb1iG0bnScGJOt/zmAl6lsOh6y,6Qp4fKIuJAkrukUvmccA8qWrezzLwFq,xwEq9YYRt37I/LTqQUWvAi1uSusxhDm,4bfMWxlbKhx/McJq/89k0SBdw.VvAzW,T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m,f/coffo2TMs2J2gq5nTOUIqH2TRAkui,q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS,7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm
instructions,Toss ingredients lightly and spoon into a butt...,Watch how to make this recipe.\nSprinkle the s...,"In a large saucepan, let the beans soak in eno...",Watch how to make this recipe.\nPreheat the ov...,Special equipment: sushi mat\nCook the brown r...,Heat a large nonstick skillet over medium-high...,Sprinkle the saffron into 1/4 cup hot water; l...,"If using top-sliced buns, spread softened butt...",Cut around the stems of the peppers (reserving...,Preheat the oven to 450 degrees F. Put the cau...,"In a large saucepan, heat 3 cups of chicken st...",Special equipment: a deep fryer with lid or a ...,MIX ingredients in 2-qt. microwaveable bowl.\n...,1. Pierce the meat all over with a fork and pl...,Cut the duck in quarters and remove the drumst...,"For Pie Dough:\nCombine f

# Cleaning Data:

## Transposing Data:

In [3]:
df_clean = df.transpose().reset_index(drop=True).drop('picture_link', axis=1)
df_clean = df_clean[['title', 'ingredients', 'instructions']]
df_clean.head()

,title,ingredients,instructions
0,Grammie Hamblet's Deviled Crab,"[1/2 cup celery, finely chopped, 1 small green...",Toss ingredients lightly and spoon into a butt...
1,Infineon Raceway Baked Beans,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Watch how to make this recipe.\nSprinkle the s...
2,Southwestern Black Bean Dip,"[1 1/2 cups dried black beans, picked over and...","In a large saucepan, let the beans soak in eno..."
3,Sour Cream Noodle Bake,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Watch how to make this recipe.\nPreheat the ov...
4,Sushi Renovation,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Special equipment: sushi mat\nCook the brown r...


## Dropping NaN Values:

In [4]:
df_clean.dropna(inplace=True)

In [5]:
df_clean = df_clean.astype(str)

In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59628 entries, 0 to 60038
Data columns (total 3 columns):
title           59628 non-null object
ingredients     59628 non-null object
instructions    59628 non-null object
dtypes: object(3)
memory usage: 1.8+ MB


In [7]:
df_clean.head()

,title,ingredients,instructions
0,Grammie Hamblet's Deviled Crab,"['1/2 cup celery, finely chopped', '1 small gr...",Toss ingredients lightly and spoon into a butt...
1,Infineon Raceway Baked Beans,"['2 pounds skirt steak, cut into 1/2-inch dice...",Watch how to make this recipe.\nSprinkle the s...
2,Southwestern Black Bean Dip,"['1 1/2 cups dried black beans, picked over an...","In a large saucepan, let the beans soak in eno..."
3,Sour Cream Noodle Bake,"['1 1/4 pounds ground chuck', 'One 15-ounce ca...",Watch how to make this recipe.\nPreheat the ov...
4,Sushi Renovation,"['1 cup rice, brown, medium-grain, cooked', '1...",Special equipment: sushi mat\nCook the brown r...


## Splitting the Data:

In [8]:
X = df_clean.drop('title', axis=1).values
y = df_clean['title'].values
print(X.shape, y.shape)

(59628, 2) (59628,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape

((44721, 2), (44721,))

In [20]:
model = Sequential()
keras.layers.Dense(2, input_shape =(2, )),
keras.layers.Dense(2, activation=tf.nn.sigmoid)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

(<keras.layers.core.Dense at 0x1a509171d0>,)

In [22]:
model.fit(X, y, epochs=150)

ValueError: Error when checking target: expected sequential_5_input to have shape (2,) but got array with shape (1,)